In [85]:
from keras import layers
from keras import models
from keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(48, 48, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [86]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 46, 46, 32)        896       
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 2048)              0         
__________

In [23]:
import os
for langs in ('NL','FR','IT','PT','ES'):
    for gend in ('M','F'):
        os.system('cp '+langs+'/imgs/'+gend+'/* ALL-DE/imgs/'+gend)

In [87]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                  )

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'ALL-DE/imgs',
        target_size=(48,48),
        batch_size=32,
        class_mode='binary',
)

validation_generator = val_datagen.flow_from_directory(
        'DE/imgs',
        target_size=(48,48),
        batch_size=20,
        class_mode='binary')



Found 9648 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [88]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=301,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=10)

Epoch 1/30
301/301 [==============================] - 17s - loss: 0.6949 - acc: 0.5242 - val_loss: 0.6994 - val_acc: 0.5050
Epoch 2/30
301/301 [==============================] - 16s - loss: 0.6895 - acc: 0.5422 - val_loss: 0.6987 - val_acc: 0.5050
Epoch 3/30
301/301 [==============================] - 16s - loss: 0.6898 - acc: 0.5404 - val_loss: 0.7037 - val_acc: 0.4900
Epoch 4/30
301/301 [==============================] - 16s - loss: 0.6885 - acc: 0.5411 - val_loss: 0.7042 - val_acc: 0.5050
Epoch 5/30
301/301 [==============================] - 16s - loss: 0.6887 - acc: 0.5440 - val_loss: 0.7025 - val_acc: 0.4950
Epoch 6/30
301/301 [==============================] - 16s - loss: 0.6889 - acc: 0.5411 - val_loss: 0.6964 - val_acc: 0.5200
Epoch 7/30
301/301 [==============================] - 16s - loss: 0.6857 - acc: 0.5484 - val_loss: 0.7046 - val_acc: 0.4850
Epoch 8/30
301/301 [==============================] - 16s - loss: 0.6863 - acc: 0.5541 - val_loss: 0.6965 - val_acc: 0.4950
Epoch 9/

In [65]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(48, 48, 3))
conv_base.summary()

58810368/58889256 [============================>.] - ETA: 0s_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (Max

In [75]:
import os
import numpy as np

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(48, 48),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features('ALL-DE/imgs/', 9648)
validation_features, validation_labels = extract_features('DE/imgs/', 200)

Found 9648 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [76]:
train_features = np.reshape(train_features, (9648, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (200, 4 * 4 * 512))

In [79]:
from keras import models
from keras import layers
from keras import optimizers

topmodel = models.Sequential()
topmodel.add(layers.Dense(128, activation='relu', input_dim=4 * 4 * 512))
topmodel.add(layers.Dropout(0.5))
topmodel.add(layers.Dense(1, activation='sigmoid'))

topmodel.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

tophistory = topmodel.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=32,
                    validation_data=(validation_features, validation_labels))

Train on 9648 samples, validate on 200 samples
Epoch 1/30
9648/9648 [==============================] - 4s - loss: 0.7178 - acc: 0.5747 - val_loss: 0.6271 - val_acc: 0.6550
Epoch 2/30
9648/9648 [==============================] - 3s - loss: 0.6407 - acc: 0.6288 - val_loss: 0.6048 - val_acc: 0.6600
Epoch 3/30
9648/9648 [==============================] - 3s - loss: 0.6253 - acc: 0.6503 - val_loss: 0.5881 - val_acc: 0.6700
Epoch 4/30
9648/9648 [==============================] - 3s - loss: 0.6160 - acc: 0.6533 - val_loss: 0.5945 - val_acc: 0.6400
Epoch 5/30
9648/9648 [==============================] - 3s - loss: 0.6100 - acc: 0.6647 - val_loss: 0.5968 - val_acc: 0.6550
Epoch 6/30
9648/9648 [==============================] - 3s - loss: 0.6063 - acc: 0.6696 - val_loss: 0.5900 - val_acc: 0.6650
Epoch 7/30
9648/9648 [==============================] - 3s - loss: 0.6011 - acc: 0.6693 - val_loss: 0.5790 - val_acc: 0.6800
Epoch 8/30
9648/9648 [==============================] - 3s - loss: 0.5990 - ac